#  Sentiment Analysis Web App
## Using PyTorch and SageMaker



## General Outline

Recall the general outline for SageMaker projects using a notebook instance.

1. Download or otherwise retrieve the data.
2. Process / Prepare the data.
3. Upload the processed data to S3.
4. Train a chosen model.
5. Test the trained model (typically using a batch transform job).
6. Deploy the trained model.
7. Use the deployed model.



## Step 1: Downloading the data


> Maas, Andrew L., et al. [Learning Word Vectors for Sentiment Analysis](http://ai.stanford.edu/~amaas/data/sentiment/). In _Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies_. Association for Computational Linguistics, 2011.

In [3]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

mkdir: cannot create directory ‘../data’: File exists
--2020-06-26 06:24:53--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  23.1MB/s    in 5.0s    

2020-06-26 06:24:58 (16.0 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



## Step 2: Preparing and Processing the data



In [4]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [5]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


Now that we've read the raw training and testing data from the downloaded dataset, we will combine the positive and negative reviews and shuffle the resulting records.

In [6]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [7]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


Now that our training and testing sets unified and prepared, we should do a quick check and see an example of the data our model will be trained on. This is generally a good idea as it allows you to see how each of the further processing steps affects the reviews and it also ensures that the data has been loaded correctly.

In [8]:
print(train_X[100])
print(train_y[100])

I still have grainy, late night, no-cable, cheap VHS dubs of this show from waaaaaayyyy back when, late-night-commercials and all, when I would stay up to whatever weird hour they would slap this show on -- just so I could tape it.<br /><br />The series wasn't really ABOUT Freddy Kreuger - only the first couple of episodes actually involved him as anything but a Rod Serling-esquire announcer. Instead, each episode was a distinct nightmare, using the traditional horror themes of horrific childhood, dating, cannibalism, dating, money, death, dating, and... hmm... dating.<br /><br />From the episode where a teenage boy accidentally says "I will love you forever" to the wrong girl, and is stuck with her (literally, at least for a moment, they grow together...), to the one where a young stewardess goes home with a strange man, only to find herself in his cabin, where he has a trophy room full of other stewardesses, and one I only vaguely remember which compared blind dates to hockey (and th

The first step in processing the reviews is to make sure that any html tags that appear should be removed. In addition we wish to tokenize our input, that way words such as *entertained* and *entertaining* are considered the same with regard to sentiment analysis.

In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

The `review_to_words` method defined above uses `BeautifulSoup` to remove any html tags that appear and uses the `nltk` package to tokenize the reviews. As a check to ensure we know how everything is working, try applying `review_to_words` to one of the reviews in the training set.

# Apply review_to_words to a review (train_X[100] or any other review)



In [10]:
review_to_words(train_X[100])


['still',
 'graini',
 'late',
 'night',
 'cabl',
 'cheap',
 'vh',
 'dub',
 'show',
 'waaaaaayyyi',
 'back',
 'late',
 'night',
 'commerci',
 'would',
 'stay',
 'whatev',
 'weird',
 'hour',
 'would',
 'slap',
 'show',
 'could',
 'tape',
 'seri',
 'realli',
 'freddi',
 'kreuger',
 'first',
 'coupl',
 'episod',
 'actual',
 'involv',
 'anyth',
 'rod',
 'serl',
 'esquir',
 'announc',
 'instead',
 'episod',
 'distinct',
 'nightmar',
 'use',
 'tradit',
 'horror',
 'theme',
 'horrif',
 'childhood',
 'date',
 'cannib',
 'date',
 'money',
 'death',
 'date',
 'hmm',
 'date',
 'episod',
 'teenag',
 'boy',
 'accident',
 'say',
 'love',
 'forev',
 'wrong',
 'girl',
 'stuck',
 'liter',
 'least',
 'moment',
 'grow',
 'togeth',
 'one',
 'young',
 'stewardess',
 'goe',
 'home',
 'strang',
 'man',
 'find',
 'cabin',
 'trophi',
 'room',
 'full',
 'stewardess',
 'one',
 'vagu',
 'rememb',
 'compar',
 'blind',
 'date',
 'hockey',
 'injuri',
 'penalti',
 'go',
 'date',
 'definit',
 'scariest',
 'thing',
 'se

In [11]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    cache_data = None
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        # Preprocess training and test data to obtain words for each review
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [12]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

Read preprocessed data from cache file: preprocessed_data.pkl


## Transform the data



### Creating a word dictionary

To begin with, we need to construct a way to map words that appear in the reviews to integers. Here we fix the size of our vocabulary (including the 'no word' and 'infrequent' categories) to be `5000` but you may wish to change this to see how it affects the model.


In [13]:
import numpy as np

def build_dict(data, vocab_size = 5000):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    
    #       sentence is a list of words.
    
    word_count = {} # A dict storing the words that appear in the reviews along with how often they occur
    for sentence in data:
        for word in sentence:
            if word not in word_count:
                word_count[word] = 1
            else:
                word_count[word] += 1
    
    #       sorted_words[-1] is the least frequently appearing word.
    
    sorted_word_freq_pairs = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
    sorted_words = [pair[0] for pair in sorted_word_freq_pairs]
    
    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [14]:
word_dict = build_dict(train_X)

In [15]:
print(list(word_dict.keys())[0:5])

['movi', 'film', 'one', 'like', 'time']


### Save `word_dict`


In [16]:
data_dir = '../data/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [17]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

### Transform the tweets



In [18]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [19]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

As a quick check to make sure that things are working as intended, check to see what one of the reviews in the training set looks like after having been processeed. Does this look reasonable? What is the length of a review in the training set?

In [20]:
print(train_X[0])

[   1 4504   28    3   34  402    1    1  216  917  332 1000 2534 2286
  120   74  152  622  394  133  402   45  575   42  418 2068 1125  397
    1  163  288   88 2108  587    1 1327  498   52   69  895    1    1
  691    1    4  120 3507  927  496   41 4584    1  985    1  314  315
   69  465  163    1  360    1    1 3237    9 2186 1416  471  129  987
  317 1296 2917  461    1    8    4   42  874  530 3150  104 1712 1096
 1266 4472 1070  288 2087   92    4   42    1  148  169  257   74   35
    1   36    1    1   95  623    1   60    1  741 3906 2187  988    1
 4909  488  196 2967 1044   40   45   27    1    1  112    1 1913    1
  231  185    1 1402 2377    1    1 2038   42  217 1125    3   27    7
  319   95    1    1  294  162    3  676 4095  473   17 1851  344  429
  141    4 1000 3085    6  828  271   41  128   52  358  152   35 2644
  611   22    3  559   36  319 1288  358   27  104   18    1   65 4356
  988   17 1034   67    1    1   74 3756  264   69    3  612    1  173
   16 

## Step 3: Upload the data to cloud


In [21]:
import pandas as pd
    
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

### Uploading the training data




In [22]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

role = sagemaker.get_execution_role()

In [23]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

## Step 4: Build and Train the PyTorch Model



In [24]:
!pygmentize train/model.py

import torch.nn as nn

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig

In [25]:
import torch
import torch.utils.data

# Read in only the first 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

# Turn the input pandas dataframe into tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

In [26]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            # TODO: Complete this train method to train the model provided.
            model.zero_grad()
            output = model.forward(batch_X)
            loss = loss_fn(output.squeeze(), batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))

In [27]:
import torch.optim as optim
from train.model import LSTMClassifier

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 100, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

train(model, train_sample_dl, 5, optimizer, loss_fn, device)

Epoch: 1, BCELoss: 0.6957642078399658
Epoch: 2, BCELoss: 0.6853172302246093
Epoch: 3, BCELoss: 0.6764956831932067
Epoch: 4, BCELoss: 0.6669299125671386
Epoch: 5, BCELoss: 0.6552591919898987


### Training the model



In [28]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.p2.xlarge',
                    hyperparameters={
                        'epochs': 10,
                        'hidden_dim': 200,
                    })

In [29]:
estimator.fit({'training': input_data})

2020-06-26 06:26:27 Starting - Starting the training job...
2020-06-26 06:26:30 Starting - Launching requested ML instances......
2020-06-26 06:27:46 Starting - Preparing the instances for training.........
2020-06-26 06:29:26 Downloading - Downloading input data......
2020-06-26 06:30:04 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-06-26 06:30:33,423 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-06-26 06:30:33,449 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-06-26 06:30:36,492 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-06-26 06:30:36,775 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-06-26 06:30:36,775 sagemaker-containers INFO     Generating setup.cfg
2020-06-26 06:30:36,775 sagemaker-contai

Model loaded with embedding_dim 32, hidden_dim 200, vocab_size 5000.
Epoch: 1, BCELoss: 0.6715278065934474
Epoch: 2, BCELoss: 0.5997781510255775
Epoch: 3, BCELoss: 0.5154977343520339
Epoch: 4, BCELoss: 0.4354088786913424
Epoch: 5, BCELoss: 0.3897775314292129
Epoch: 6, BCELoss: 0.35633550310621454
Epoch: 7, BCELoss: 0.3284029480145902
Epoch: 8, BCELoss: 0.33173333260477805
Epoch: 9, BCELoss: 0.2894860943969415

2020-06-26 06:34:00 Uploading - Uploading generated training modelEpoch: 10, BCELoss: 0.289119119242746
2020-06-26 06:33:58,858 sagemaker-containers INFO     Reporting training SUCCESS

2020-06-26 06:34:08 Completed - Training job completed
Training seconds: 282
Billable seconds: 282


## Step 5: Testing the model

As mentioned at the top of this notebook, we will be testing this model by first deploying it and then sending the testing data to the deployed endpoint. We will do this so that we can make sure that the deployed model is working correctly.

## Step 6: Deploy the model for testing



In [30]:
#Deploy the trained model
predictor = estimator.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')


---------------!

In [31]:
predictor.endpoint


'sagemaker-pytorch-2020-06-26-06-26-26-932'

## Step 7 - Use the model for testing

Once deployed, we can read in the test data and send it off to our deployed model to get some results. Once we collect all of the results we can determine how accurate our model is.

In [32]:
test_X = pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1)

In [33]:
# We split the data into chunks and send each chunk seperately, accumulating the results.

def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        predictions = np.append(predictions, predictor.predict(array))
    
    return predictions

In [34]:
predictions = predict(test_X.values)
predictions = [round(num) for num in predictions]

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.84904

### More testing



In [36]:
test_review = 'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'

### Converting test_review into a form usable by the model and save the results in test_data

In [37]:
test_data, leng  = convert_and_pad(word_dict, review_to_words(test_review))
len_test_data = np.array([np.array([leng] + test_data)])

### Delete the endpoint


## Step 6 (again) - Deploy the model for the web app

Now that we know that our model is working, it's time to create some custom inference code so that we can send the model a review which has not been processed and have it determine the sentiment of the review.

### Writing inference code



In [44]:
!pygmentize serve/predict.py

import argparse
import json
import os
import pickle
import sys
import sagemaker_containers
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

from model import LSTMClassifier

from utils import review_to_words, convert_and_pad

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LSTMClassifier(model_info['embedding_dim'], model_info['hidden_dim'], model_info['vocab_size'])

    # Load the store model parameters.
    model_path = os.path.join(mod

### Deploying the model



In [46]:
model_data = estimator.model_data
model_data

's3://sagemaker-us-east-1-131217235708/sagemaker-pytorch-2020-06-26-06-26-26-932/output/model.tar.gz'

In [47]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

-----------------!

### Testing the model


In [48]:
import glob

def test_reviews(data_dir='../data/aclImdb', stop=250):
    
    results = []
    ground = []
    
    # We make sure to test both positive and negative reviews    
    for sentiment in ['pos', 'neg']:
        
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        
        files_read = 0
        
        print('Starting ', sentiment, ' files')
        
        # Iterate through the files and send them to the predictor
        for f in files:
            with open(f) as review:
                # First, we store the ground truth (was the review positive or negative)
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                # Read in the review and convert to 'utf-8' for transmission via HTTP
                review_input = review.read().encode('utf-8')
                # Send the review to the predictor and store the results
                pred = predictor.predict(review_input)
                results.append(float(pred))
                
            # Sending reviews to our endpoint one at a time takes a while so we
            # only send a small number of reviews
            files_read += 1
            if files_read == stop:
                break
            
    return ground, results


In [49]:
ground, results = test_reviews()

Starting  pos  files
Starting  neg  files


In [50]:
from sklearn.metrics import accuracy_score
accuracy_score(ground, results)

0.848

In [51]:
predictor.predict(test_review)

b'1.0'

## Step 7 (again): Use the model for the web app





In [52]:
predictor.endpoint

'sagemaker-pytorch-2020-06-26-06-54-52-723'

### Delete the endpoint



In [64]:
predictor.delete_endpoint()